In [3]:
#import sys
#sys.path.append('../../earth_engine_MP/')
import ee
from google.oauth2 import service_account

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import geetools
import eemont
import geemap
from ee_extra.TimeSeries.core import getTimeSeriesByRegion

from plot_ee_fctns import get_image_index, box_around_point, mask_other, initialise_comparison
from masks import mask_MODIS_clouds, MODIS_Mask_QC, mask_s2_clouds, mask_s2_clouds_collection, csPlus_mask_collection, MODIS_mask_clouds_250m
import masks

from scipy.interpolate import griddata

# Path to the private key file
key_path = 'Access/ee-martinparker637-e68fde65abb4.json'

# Load the service account credentials
credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=['https://www.googleapis.com/auth/earthengine'])

# Initialize Earth Engine with the service account credentials
ee.Initialize(credentials)
root_directory = ''

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


In [ ]:
ds = xr.open_dataset('Saved_files/gridded_sat/Kenya/LANDSAT/Kenya_LANDSAT_CMG_2002_07.nc')
ds["time"] = ds["time"].dt.floor("D")

In [ ]:
ds["time"] = ds["time"].dt.floor("D")
daily_index = pd.date_range(ds.time.min().values, ds.time.max().values, freq='D')
ds = ds.groupby('time').max(skipna=True)#ds.drop_duplicates(dim='time')#
ds_daily = ds.reindex(time=daily_index)
ds_filled = ds_daily.interpolate_na(dim='time', method='linear')#, fill_value="extrapolate")#ds_daily.ffill(dim='time')#

In [ ]:
#fig, ax = plt.subplots(1, 2)
#ind = 22
#ds['SR_B4'].sel(time = ds['time'][ind]).plot(ax = ax[0])
#ds_daily['SR_B4'].sel(time = ds['time'][ind]).plot(ax = ax[1])

In [ ]:
ds_filled['NDVI'].isel(time = 10).T.plot()

In [ ]:
ds_daily.ffill(dim='time').dropna(dim='lon', how='all').dropna(dim='lat', how='all').dropna(dim='time', how='all')

In [ ]:
ds = xr.open_dataset('Saved_files/gridded_sat/Kenya/MODIS/Kenya_MODIS_CMG_VIs_2000_2010.nc')

In [ ]:
def open_and_combine_months(year, miss_months = []):
    ds = xr.open_dataset(f'Saved_files/gridded_sat/Kenya/LANDSAT/Kenya_LANDSAT_CMG_{year}_01.nc')
    ds = daily_mosaic(ds)
    for month in range(2, 13):
        if month in miss_months:
            continue
        print(month)
        month_name = str(month).zfill(2)
        ds_append = xr.open_dataset(f'Saved_files/gridded_sat/Kenya/LANDSAT/Kenya_LANDSAT_CMG_{year}_{month_name}.nc')
        ds_append = daily_mosaic(ds_append)
        ds = xr.concat([ds, ds_append], 
                       dim = 'time', 
                       combine_attrs="override", 
                       coords="minimal",
                       compat="override")
        print(ds_append.dims)
    return ds

In [ ]:
def open_and_combine_months_debug(year, miss_months = []):
    ds = xr.open_dataset(f'Saved_files/gridded_sat/Kenya/LANDSAT/Kenya_LANDSAT_CMG_{year}_01.nc')
    ds = daily_mosaic(ds)
    for month in range(2, 13):
        if month in miss_months:
            continue
        print(month)
        month_name = str(month).zfill(2)
        ds_append = xr.open_dataset(f'Saved_files/gridded_sat/Kenya/LANDSAT/Kenya_LANDSAT_CMG_{year}_{month_name}.nc')
        ds_append = daily_mosaic(ds_append)
        for var in ds.data_vars:
            if not ds[var].equals(ds_append[var]):
                print(f"Variable {var} differs in month {month}")
        ds = xr.concat([ds, ds_append], dim = 'time', combine_attrs="override", compat="override")
        print(ds_append.dims)
    return ds

In [ ]:
def check_coords2(file_list):
    for f in file_list:
        with xr.open_dataset(f) as ds:
            print(f"\n{f}")
            for c in ds.coords:
                print(f"  {c}: shape={ds[c].shape}, dtype={ds[c].dtype}")

In [ ]:
def check_coords3(file_list):
    ref = xr.open_dataset(file_list[0])
    for f in file_list[1:]:
        ds = xr.open_dataset(f)
        for v in ref.data_vars:
            if v not in ds:
                print(f"{f} missing var {v}")
            else:
                if ref[v].shape != ds[v].shape:
                    print(f"{f}: {v} shape mismatch {ref[v].shape} vs {ds[v].shape}")
        ds.close()
    ref.close()

In [ ]:
def check_coords(file_list):
    """
    Check whether all NetCDF files in file_list share the same coordinates.
    Prints mismatches if any.
    """
    ref = xr.open_dataset(file_list[0])
    ref_coords = {k: ref.coords[k].values for k in ref.coords}

    for f in file_list[1:]:
        ds = xr.open_dataset(f)
        for k in ref.coords:
            if k == 'time':
                continue
            if k not in ds.coords:
                print(f"{f}: missing coord {k}")
                continue
            if not (ref_coords[k] == ds.coords[k].values).all():
                print(f"{f}: coordinate {k} differs")
        # also check for extra coords
        for k in ds.coords:
            if k not in ref_coords:
                print(f"{f}: has extra coord {k}")
        ds.close()
    ref.close()

In [ ]:
def combine_months2(year, miss_months):
    month_list = list(range(1, 13))
    for n in miss_months:
        month_list.remove(n)
    file_list = [f'Saved_files/gridded_sat/Kenya/LANDSAT/Kenya_LANDSAT_CMG_{year}_{str(n).zfill(2)}.nc' for n in month_list]
    datasets = [clean(xr.open_dataset(f)) for f in file_list]
    ds = xr.concat(
        datasets,
        dim="time",
        compat="override",
        coords="minimal",
        combine_attrs="override"
    )
    ds = daily_mosaic(ds)
    return ds

In [ ]:
for i, year in enumerate(np.arange(2000, 2025)):
    print(year)
    if year == 2003:
        miss_months = [6]
    else:
        miss_months = []
    ds_new = combine_months2(year, miss_months = miss_months)
    ds_new.to_netcdf(f'Saved_files/gridded_sat/Kenya/LANDSAT_PROCESSED/Kenya_LANDSAT_CMG_{year}.nc')

In [ ]:
ds_list=[]
for i, year in enumerate(np.arange(2000, 2025)):
    ds_list.append(xr.open_dataset(f'Saved_files/gridded_sat/Kenya/LANDSAT_PROCESSED/Kenya_LANDSAT_CMG_{year}.nc'))
    